In [85]:
import os
from dotenv import load_dotenv
import gradio as gr
import requests

In [86]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import HumanMessage, AIMessage, ToolMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.utils.function_calling import convert_to_openai_tool
from langchain_core.runnables import RunnablePassthrough

In [87]:
MODEL = 'gemini-2.0-flash'

# Load environment variables from .env file
load_dotenv(override=True)
GOOGLE_GEMINI_KEY = os.environ['GOOGLEAI_API_KEY']
WEATHER_API_KEY = os.environ['WEATHER_API_KEY'] # https://www.weatherapi.com/

In [93]:
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location."""
    url = f'http://api.weatherapi.com/v1/current.json?key={WEATHER_API_KEY}&q={location}&aqi=no'
    response = requests.get(url)
    if response.ok:
        data = response.json()
        # Get current weather temperature in celsius
        return f"Current temperature for {location} is {data['current']['temp_c']}"
    else:
        return f"Não foi possível obter a previsão do tempo para {location}C."

tools = [convert_to_openai_tool(get_current_weather)]

In [89]:
# Configure Gemini LLM
llm = ChatGoogleGenerativeAI(model=MODEL, google_api_key=GOOGLE_GEMINI_KEY)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant"),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
    ]
)

llm_with_tools = llm.bind_tools(tools=tools)
chain = prompt | llm_with_tools

In [97]:
def chat(message, history):
    # Converta o histórico do Gradio para o formato LangChain
    chat_history = []
    for human, ai in history:
        chat_history.append(HumanMessage(content=human))
        chat_history.append(AIMessage(content=ai))

    # Adicione a nova mensagem
    chat_history.append(HumanMessage(content=message))

    # Invoque a cadeia
    response = chain.invoke({"input": message, "chat_history": chat_history})

    # Verifique se há chamadas de ferramentas
    if hasattr(response, 'tool_calls') and response.tool_calls:
        for tool_call in response.tool_calls:
            function_name = tool_call['name']
            args = tool_call['args']
            if function_name == 'get_current_weather':
                weather_result = get_current_weather(**args)
                return weather_result
    else:
        # Se não houver tool_calls, retorne o conteúdo da resposta
        return response.content

# # Teste a função
# query = "how is the weather in Bauru?"
# history = [
#     ("Hello!", "Hi! How can I assist you today?"),
# ]
# result = chat(query, history)
# print("\nAnswer:", result)
app = gr.ChatInterface(
    fn=chat,
    type="messages",
).launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/opt/miniconda3/envs/ai-study/lib/python3.10/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
  File "/opt/miniconda3/envs/ai-study/lib/python3.10/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
  File "/opt/miniconda3/envs/ai-study/lib/python3.10/site-packages/gradio/blocks.py", line 2136, in process_api
    result = await self.call_function(
  File "/opt/miniconda3/envs/ai-study/lib/python3.10/site-packages/gradio/blocks.py", line 1660, in call_function
    prediction = await fn(*processed_input)
  File "/opt/miniconda3/envs/ai-study/lib/python3.10/site-packages/gradio/utils.py", line 851, in async_wrapper
    response = await f(*args, **kwargs)
  File "/opt/miniconda3/envs/ai-study/lib/python3.10/site-packages/gradio/chat_interface.py", line 876, in _submit_fn
    response = await anyio.to_thread.run_sync(
  